In [1]:
from __future__ import division
import re
import nltk
import logging
import os
import string
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from nltk.corpus import WordNetCorpusReader
from emotion import Emotion as Emo
from senpy.plugins import EmotionPlugin, SenpyPlugin, ShelfMixin
from senpy.models import Results, EmotionSet, Entry, Emotion


In [5]:
# wn16 = WordNetCorpusReader(os.path.abspath("{0}".format(self._wn16_path)), nltk.data.find(self._wn16_path))
wn16_path = "/data/emotion-wnaffect/wordnet1.6/dict"
wn16 = WordNetCorpusReader(wn16_path, nltk.data.find(wn16_path))
#  the second param should be an omw_reader !! (:

In [ ]:
wn16.synsets("sad")

In [3]:
nltk.download("omw")

[nltk_data] Downloading package omw to /home/ianwoo/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

## testing nltk.corpus.stopwords for multilingual text

In [40]:
stopwords.words("README")

['Stopwords Corpus',
 'This corpus contains lists of stop words for several languages.  These',
 'are high-frequency grammatical words which are usually ignored in text',
 'retrieval applications.',
 'They were obtained from:',
 'http://anoncvs.postgresql.org/cvsweb.cgi/pgsql/src/backend/snowball/stopwords/',
 'The English list has been augmented',
 'https://github.com/nltk/nltk_data/issues/22',
 'The German list has been corrected',
 'https://github.com/nltk/nltk_data/pull/49',
 'A Kazakh list has been added',
 'https://github.com/nltk/nltk_data/pull/52']

## nltk.corpus.wn and multilingual wn experiments

In [3]:
from nltk.corpus import wordnet as wn
print(" ".join(l for l in wn.langs() if len(l)==3))
print(sum(1 for l in wn.langs() if len(l)==3))
print(" ".join(sorted(l for l in wn.langs() if len(l)==2)))
print(sum(1 for l in wn.langs() if len(l)==2))
# print()
# print(", ".join(sorted(l for l in wn.langs())))

eng als arb bul cat cmn dan ell eus fas fin fra glg heb hrv ind ita jpn nno nob pol por qcn slv spa swe tha zsm
28
bg cs da de el es et fi fr ga hr hu it lt lv mt nl pl pt ro sk sl sv
23


In [33]:
from nltk.corpus import wordnet as wn
syn = wn.synsets("sad")[0]
"%d-%s"%(syn.offset(), syn.pos())

'1361863-a'

In [48]:
syns = wn.synsets("hlad",lang='cs')
print(len(syns), type(syns))
print(len(set(syn.offset() for syn in syns)))

31 <class 'list'>
25


In [2]:
Emo.emotions = {}
# This is not changed from the original
def _load_emotions(self, hierarchy_path):
    """Loads the hierarchy of emotions from the WordNet-Affect xml."""

    tree = ET.parse(hierarchy_path)
    root = tree.getroot()
    for elem in root.findall("categ"):
        name = elem.get("name")
        if name == "root":
            Emo.emotions["root"] = Emo("root")
        else:
            Emo.emotions[name] = Emo(name, elem.get("isa"))
_load_emotions(None, "wn-affect-1.1/a-hierarchy.xml")

In [14]:
mapping_wn16_30_path = "mapping-16-30"
wn16_30 = {}
pos_mapping_suffixes = {"a":"adj", "r":"adv", "n":"noun", "v":"verb"}
# for pos in ("a", "r", "n", "v"):
for pos,suffix in pos_mapping_suffixes.items():
    mapping = {}
    path = os.path.join(mapping_wn16_30_path,"wn16-30.{0}".format(suffix))
    with open(path) as f:
        for l in f:
            l = l.split()
            # get mapping with greatest probability
            target, prob = 0, 0
            for t,p in zip(l[1::2],l[2::2]):
                p = float(p.strip())
                if p>prob:
                    target,prob = t,p
            mapping[int(l[0])] = int(target)
    wn16_30[pos] = mapping
    

def _convert_wn_16_30(self, pos, wn16_offset):
    return wn16_30[pos][wn16_offset]

In [4]:
    unconvertedSynsetIds = []

    def _load_synsets(self, synsets_path):
        """Returns a dictionary synset offset -> emotion (int -> str)."""
        # """Returns a dictionary POS tag -> synset offset -> emotion (str -> int -> str)."""
        def convertId(wn16id):
            pos_char,wn16_offset = wn16id.split("#")
            try:
                return "{0}-{1}".format(wn16_30[pos_char][int(wn16_offset)], pos_char)
            except KeyError:
                # unconvertedSynsetIds.append((wn16id, pos_char, wn16_offset))
                return None

        tree = ET.parse(synsets_path)
        root = tree.getroot()

        synsets = {}
        pending = []
        for pos in ["noun", "adj", "verb", "adv"]:
            for elem in root.findall(".//{0}-syn-list//{0}-syn".format(pos)):
                idd = elem.get("id")
                wn30_id = convertId(idd)
                if wn30_id is None: 
                    continue
                if elem.get("categ"):
                    synsets[wn30_id] = Emo.emotions[elem.get("categ")] if elem.get("categ") in Emo.emotions else None
                elif elem.get("noun-id"):
                    try:
                        synsets[wn30_id] = synsets[convertId(elem.get("noun-id"))]
                    except KeyError:
                        newid = convertId(elem.get("noun-id"))
                        if newid is not None:
                            pending.append((wn30_id, newid))
        for (wn30_id, noun_id) in pending:
            try:
                synsets[wn30_id] = synsets[noun_id]
            except KeyError:
                print("trouble with id",wn30_id, noun_id)
                raise
        return synsets


In [5]:
wnaff = _load_synsets(None, "wn-affect-1.1/a-synsets.xml")

In [17]:
1361863 in wnaff

False

In [19]:
list(wnaff.keys())[:5]

['7513508-n', '822907-a', '7531998-n', '173498-r', '7517292-n']

In [6]:
unconvertedSynsetIds

[('a#00196250', 'a', '00196250')]

The one id that doesn't convert to wn3.0 is **a#00196250** but its wn-affect category is **calmness**, which currently has no Ekman emotion associated.

In [7]:
Emo.printTree()

     ┌physical-state 1
     ├behaviour 1
     ├trait 1
     ├sensation 1
     ├situation 1┐
     │         └emotion-eliciting-situation 2
     ├signal 1┐
     │      └edonic-signal 2
 root 0┤
     │            ┌cognitive-state 2
     │            ├cognitive-affective-state 2
     └mental-state 1┤
                  │               ┌mood 3
                  └affective-state 2┤
                                  │                              ┌emotionlessness 6
                                  │                       ┌apathy 5┤
                                  │                       │      └neutral-languor 6
                                  │       ┌neutral-emotion 4┤
                                  │       │               └neutral-unconcern 5┐
                                  │       │                                 │            ┌distance 7
                                  │       │                                 └indifference 6┤
                                  │       │      

In [13]:
set(["foo","fum"])

{'foo', 'fum'}

In [11]:
for level in (5,):
    for name, emo in Emo.emotions.items():
        if emo.level == level:
            print("%-28s %2d %s"%(emo.name, emo.level, emo.parent))

fearlessness                  5 positive-emotion
liking                        5 positive-emotion
love                          5 positive-emotion
sadness                       5 negative-emotion
self-pride                    5 positive-emotion
shame                         5 negative-emotion
negative-fear                 5 negative-emotion
anxiety                       5 negative-emotion
calmness                      5 positive-emotion
compassion                    5 negative-emotion
levity                        5 positive-emotion
ambiguous-fear                5 ambiguous-emotion
humility                      5 negative-emotion
pensiveness                   5 ambiguous-emotion
daze                          5 negative-emotion
ambiguous-agitation           5 ambiguous-emotion
positive-hope                 5 positive-emotion
thing                         5 ambiguous-emotion
positive-expectation          5 positive-emotion
despair                       5 negative-emotion
neutral-unconcer

In [118]:
categories = set()
for idd,emo in wnaff.items():
    #print(idd,":",emo)
    categories.add(str(emo))
    if str(emo) == 'surprise':
        print(emo.name, emo.level, emo.parent, [e.name for e in emo.children])
'surprise' in categories

surprise 7 astonishment []
surprise 7 astonishment []
surprise 7 astonishment []
surprise 7 astonishment []
surprise 7 astonishment []
surprise 7 astonishment []
surprise 7 astonishment []


True

# Below uses pattern package - Python 2 only!!

In [17]:
from pattern.en import parse

text = "I had a tomato at the shop."
sentencesP = parse(text,lemmata=True)
print sentencesP
sentencesP.split??
sentences = sentencesP.split()
print sentences
print
for s in sentences:
    print
    for w in s:
        for p in w:
            print "%7s"%p,
        print
print

I/PRP/B-NP/O/i had/VBD/B-VP/O/have a/DT/B-NP/O/a tomato/NN/I-NP/O/tomato at/IN/B-PP/B-PNP/at the/DT/B-NP/I-PNP/the shop/NN/I-NP/I-PNP/shop ././O/O/.
[[[u'I', u'PRP', u'B-NP', u'O', u'i'], [u'had', u'VBD', u'B-VP', u'O', u'have'], [u'a', u'DT', u'B-NP', u'O', u'a'], [u'tomato', u'NN', u'I-NP', u'O', u'tomato'], [u'at', u'IN', u'B-PP', u'B-PNP', u'at'], [u'the', u'DT', u'B-NP', u'I-PNP', u'the'], [u'shop', u'NN', u'I-NP', u'I-PNP', u'shop'], [u'.', u'.', u'O', u'O', u'.']]]


      I     PRP    B-NP       O       i
    had     VBD    B-VP       O    have
      a      DT    B-NP       O       a
 tomato      NN    I-NP       O  tomato
     at      IN    B-PP   B-PNP      at
    the      DT    B-NP   I-PNP     the
   shop      NN    I-NP   I-PNP    shop
      .       .       O       O       .

